In [1]:
import pandas as pd 
import numpy as np 
import tensorflow.compat.v2 as tf
#import tensorflow_datasets as tfds

tf.enable_v2_behavior()

from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

from tensorflow.python.compiler.mlcompute import mlcompute
mlcompute.set_mlc_device(device_name='gpu')

In [2]:
from tensorflow.compat.v2.keras import layers,Sequential
from tensorflow.compat.v2.keras import Model
from tensorflow.compat.v2 import random
from tensorflow.compat.v2.keras.losses import BinaryCrossentropy,KLDivergence
from tensorflow.compat.v2 import GradientTape
from tensorflow.compat.v2 import keras
import datetime

In [3]:
wine_df = pd.read_csv('data/wine.csv').drop(['quality','wine_type'],axis=1) 
wine_array = wine_df.to_numpy()
#dropping catagorical features, quality and type
print(np.shape(wine_array))


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(wine_array)
xscaled = scaler.transform(wine_array)

(6497, 11)


In [117]:
#using https://www.tensorflow.org/tutorials/generative/cvae
class VAE_Simple(Model): 
    
    def __init__(self,latent_dims):
        super(VAE_Simple, self).__init__()
        self.latent_dims = latent_dims
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(11)),
            layers.Dense(400,  activation='relu'),
            layers.Dense(300,  activation='relu'),
            layers.Dense(200,  activation='relu'),
            layers.Dense(100,  activation='relu'),
            # No activation
            layers.Dense(self.latent_dims + self.latent_dims),
        ])
        
        
        self.decoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=(latent_dims,)),
            layers.Dense(100,  activation='relu'),
            layers.Dense(200,  activation='relu'),
            layers.Dense(300,  activation='relu'),
            layers.Dense(400,  activation='relu'),
            #activation as data scaled on [0,1]
            layers.Dense(11),  
        ])
        
    #@tf.function
    def encode(self,X):
        mean, logvar = tf.split(self.encoder(X), num_or_size_splits=2, axis=1)
        return mean, logvar
      
      
    def sample(self, eps=None):
        if eps is None:
          eps = tf.random.normal(shape=(100, self.latent_dims))
        return self.decode(eps)

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        eps = random.normal(shape=mean.shape)
        z   = eps * tf.exp(logvar * .5) + mean
        return z

    def decode(self, z):
        probs = self.decoder(z)
        return probs

model = VAE_Simple(10)
mean,logvar = model.encode(xscaled)
z           = model.reparameterize(mean,logvar)
probs = model.decode(z)

In [98]:
import matplotlib.pyplot as plt

In [118]:
def loss_function(x,reconx,mu,logvar): 
    """ 
    Loss function using BCE and KL divergence in analytical form
    """
    #reduction=tf.keras.losses.Reduction.SUM
    BCE  =  tf.nn.sigmoid_cross_entropy_with_logits(logits=reconx, labels=x)
    KL   = -0.5*tf.math.reduce_sum(1+logvar-tf.math.square(mu)-tf.math.exp(logvar))
    #KL   /= tf.dtypes.cast(tf.shape(x)[0]*11,tf.float32)
    print(BCE(x,reconx).numpy())
    loss = BCE(x,reconx) # + KL
    return loss

In [119]:
optimizer = tf.keras.optimizers.Adam(1e-4)
model = VAE_Simple(100)
loss_store = []
for _ in range(100):
    with GradientTape() as tape: 
        mean,logvar = model.encode(xscaled)
        z           = model.reparameterize(mean,logvar)
        probs       = model.decode(z)
        loss_value  = loss_function(xscaled,probs,mean,logvar)
    
        
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    loss_store.append(loss_value.numpy())
plt.plot(loss_store)
plt.show()

InvalidArgumentError: cannot compute _MLCEagerMul as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:Mul]

In [101]:
model.sample()

<tf.Tensor: shape=(100, 11), dtype=float32, numpy=
array([[0.10180381, 0.10065804, 0.10002148, ..., 0.09767394, 0.09930038,
        0.10037296],
       [0.0889376 , 0.08991352, 0.08895203, ..., 0.09262154, 0.08899774,
        0.0885021 ],
       [0.0855319 , 0.08774209, 0.08647577, ..., 0.08727247, 0.08433586,
        0.08379878],
       ...,
       [0.09498784, 0.09540915, 0.09200555, ..., 0.0892857 , 0.08955231,
        0.08813804],
       [0.10051127, 0.1033577 , 0.10337511, ..., 0.09821001, 0.09895917,
        0.10021628],
       [0.08958513, 0.0899066 , 0.09171642, ..., 0.09184995, 0.09305593,
        0.09275003]], dtype=float32)>